# IRS `outflow` migration at the state level

#### Import Python tools

In [2]:
%load_ext lab_black

In [3]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_grid as altgrid
import numpy as np
import us

In [4]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [5]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Read [migration data](https://www.irs.gov/statistics/soi-tax-stats-data-by-geographic-area) from IRS (processed in 00, 01 notebooks)

In [6]:
inflow = pd.read_csv("data/processed/irs-state-inflow-migration-2019_20_grouped.csv")

In [7]:
inflow.head()

,destination_fips,destination_state,returns,inflow_taxpayers,population,new_inflow_taxpayers_per_1k,new_inflow_taxpayers_share_all_movers
0,12,Florida,337589,623700,21216924,29.40,9.08
1,48,Texas,280783,561736,28635442,19.62,8.17
2,6,California,248447,412714,39346023,10.49,6.01
3,37,North Carolina,157714,300941,10386227,28.98,4.38
4,13,Georgia,139003,273218,10516579,25.98,3.98


In [9]:
outflow = pd.read_csv("data/processed/irs-state-outflow-migration-2019_20_grouped.csv")

In [10]:
outflow.head()

,origin_fips,origin_state,returns,outflow_taxpayers,population,lost_outflow_taxpayers_per_1k,lost_outflow_taxpayers_share_all_movers
0,6,California,365922,676058,39346023,17.18,9.86
1,36,New York,278489,477131,19514849,24.45,6.96
2,12,Florida,256188,456993,21216924,21.54,6.66
3,48,Texas,218116,428286,28635442,14.96,6.25
4,17,Illinois,151628,272632,12716164,21.44,3.98


In [14]:
net = pd.merge(
    inflow[["destination_fips", "destination_state", "inflow_taxpayers"]],
    outflow[["origin_fips", "origin_state", "outflow_taxpayers"]],
    left_on="destination_fips",
    right_on="origin_fips",
)

In [16]:
net["net_taxpayers"] = net["inflow_taxpayers"] - net["outflow_taxpayers"]

In [19]:
net.sort_values("net_taxpayers", ascending=False)

,destination_fips,destination_state,inflow_taxpayers,origin_fips,origin_state,outflow_taxpayers,net_taxpayers
0,12,Florida,623700,12,Florida,456993,166707
1,48,Texas,561736,48,Texas,428286,133450
5,4,Arizona,247624,4,Arizona,167591,80033
3,37,North Carolina,300941,37,North Carolina,232767,68174
13,45,South Carolina,174073,45,South Carolina,120081,53992
11,47,Tennessee,201180,47,Tennessee,156078,45102
4,13,Georgia,273218,13,Georgia,236144,37074
28,16,Idaho,87032,16,Idaho,50377,36655
18,32,Nevada,130679,32,Nevada,102606,28073
25,49,Utah,93926,49,Utah,76769,17157


---

## Exports

In [28]:
# df.to_csv("data/processed/irs-state-outflow-migration-2019_20.csv", index=False)
# origins_grouped.to_csv(
#     "data/processed/irs-state-outflow-migration-2019_20_grouped.csv", index=False
# )